This explains how I implemented the image transformations in `ImageLab/src/components/ImageCanvas.tsx`. I present each transform as an algorithm, prove its correctness, and explain my design decisions.

## 1. Color Space and Design

### 1.1 sRGB

All modern displays, web browsers, and the HTML Canvas API work in sRGB color space. sRGB values are gamma-encoded. They don't represent linear light intensity, but rather perceptually-uniform brightness levels optimized for human vision.

This means that...
- Values range from 0 to 255 for each channel (R, G, B)
- The relationship to linear light is non-linear: $R_{linear} \approx (R_{sRGB}/255)^{2.2}$
- Value 128 appears as perceptual "middle gray" on screen (not 50% light intensity)
- Gamma encoding makes efficient use of 8 bits by allocating more values to darker tones where human eyes are more sensitive

### 1.2 Design Decisions

I chose to perform all transformations in sRGB space (without linearization because):

1. Linearizing and re-encoding requires power operations per pixel which are expensive and computation matters as we are working with large images
2. For real-time image editing, gamma-space operations provide visually acceptable results
3. Canvas ImageData provides sRGB values

This means most transforms work on gamma-encoded values. I've ensured each algorithm is mathematically sound in this space.

As I am most comfortable with Python, I originally thought it would be most computationally efficient to leverage libraries for the matrix operations. However, when looking through the documentation for libraries like ml-matrix or numeric.js, I noticed they require individual function calls for each operation. Unlike NumPy, which delegates to optimized C/Fortran code and processes batched operations in compiled code, JavaScript matrix libraries execute in the JavaScript runtime. For our use case—applying a 3×3 transformation to 2+ million pixels—this means 2+ million separate function calls, each with JavaScript overhead, object allocations, and garbage collection pressure.

Therefore, I write all matrix operations inline by extracting matrix elements and computing per pixel.

### 1.3 Mathematical Notation

- $\mathbf{r} = [R, G, B]^T$ represents a pixel in gamma-encoded sRGB space, where each component is in $[0, 255]$
- $I$ denotes the $3 \times 3$ identity matrix
- $M$ denotes a $3 \times 3$ transformation matrix
- $\mathbf{o}$ denotes a $3 \times 1$ offset vector
- An affine transform has the form: $\mathbf{r}' = M\mathbf{r} + \mathbf{o}$
- All operations include implicit clamping to $[0, 255]$




## 2. Brightness Adjustment

**What I wanted:** A transform that shifts all color channels by the same amount, making the image uniformly lighter or darker.

### Algorithm 2.1: Brightness Transform

```
Input: pixel r = [R, G, B]ᵀ, brightness offset b ∈ ℝ
Output: transformed pixel r' = [R', G', B']ᵀ

1. R' ← clamp(R + b, 0, 255)
2. G' ← clamp(G + b, 0, 255)  
3. B' ← clamp(B + b, 0, 255)
4. return [R', G', B']ᵀ
```

**Affine form:** $\mathbf{r}' = I\mathbf{r} + b\mathbf{1}$ where $\mathbf{1} = [1, 1, 1]^T$

**Implementation:** See `buildBrightnessMatrix()` at lines 40-49 in `ImageCanvas.tsx`:
```typescript
const buildBrightnessMatrix = (value: number): { matrix: number[]; offset: number[] } => {
  const matrix = [1, 0, 0, 0, 1, 0, 0, 0, 1]; // Identity matrix
  const offset = [value, value, value]; // Uniform offset
  return { matrix, offset };
};
```

### Proof of Correctness

**Claim:** This transform shifts all channels uniformly while preserving color relationships.

**Proof:**
For any two pixels $\mathbf{r}_1, \mathbf{r}_2$:
$$\mathbf{r}'_1 - \mathbf{r}'_2 = (I\mathbf{r}_1 + b\mathbf{1}) - (I\mathbf{r}_2 + b\mathbf{1}) = I(\mathbf{r}_1 - \mathbf{r}_2) = \mathbf{r}_1 - \mathbf{r}_2$$

The difference between any two pixels remains constant, preserving relative color relationships. The transform is a pure translation in RGB space. 


## 3. Contrast Adjustment

**What I wanted:** A transform that scales distances from middle gray (128), making dark regions darker and bright regions brighter while keeping middle gray fixed.

### Algorithm 3.1: Contrast Transform

```
Input: pixel r = [R, G, B]ᵀ, contrast factor c ∈ ℝ⁺
Output: transformed pixel r'

1. For each channel k ∈ {R, G, B}:
2.   k' ← 128 + c(k - 128)
3.   k' ← clamp(k', 0, 255)
4. return r'
```

**Affine form:** $\mathbf{r}' = cI\mathbf{r} + 128(1-c)\mathbf{1}$

**Implementation:** See `buildContrastMatrix()` at lines 53-62 in `ImageCanvas.tsx`:
```typescript
const buildContrastMatrix = (value: number): { matrix: number[]; offset: number[] } => {
  const matrix = [value, 0, 0, 0, value, 0, 0, 0, value]; // Scale by c
  const offset = [128 * (1 - value), 128 * (1 - value), 128 * (1 - value)];
  return { matrix, offset };
};
```

### Proof of Correctness

**Claim:** Middle gray (128) is a fixed point, and distances from middle gray scale by factor $c$.

**Proof:**
Let $\mathbf{g} = [128, 128, 128]^T$ represent middle gray.

**(1) Fixed point property:**
$$\mathbf{g}' = cI\mathbf{g} + 128(1-c)\mathbf{1} = c \cdot 128 + 128(1-c) = 128 = \mathbf{g}$$

**(2) Distance scaling:**
For any pixel $\mathbf{r}$, the distance from middle gray after transformation is:
$$\mathbf{r}' - \mathbf{g} = (cI\mathbf{r} + 128(1-c)\mathbf{1}) - 128\mathbf{1} = c(\mathbf{r} - 128\mathbf{1}) = c(\mathbf{r} - \mathbf{g})$$

Thus distances from middle gray scale by exactly $c$. 



## 4. Saturation Adjustment

**What I wanted:** A transform that blends each color with its grayscale equivalent, controlling color intensity while preserving perceived brightness.

### Algorithm 4.1: Saturation Transform

```
Input: pixel r = [R, G, B]ᵀ, saturation factor s ∈ ℝ
Output: transformed pixel r'

1. L ← 0.2126·R + 0.7152·G + 0.0722·B    // Rec.709 luminance
2. gray ← [L, L, L]ᵀ
3. r' ← gray·(1 - s) + r·s               // Linear interpolation
4. return clamp(r', 0, 255)
```

**Affine form:** This can be expressed as $\mathbf{r}' = M_s\mathbf{r}$ where:

$$M_s = \begin{bmatrix}
w_R + (1-w_R)s & w_G(1-s) & w_B(1-s) \\
w_R(1-s) & w_G + (1-w_G)s & w_B(1-s) \\
w_R(1-s) & w_G(1-s) & w_B + (1-w_B)s
\end{bmatrix}$$

where $(w_R, w_G, w_B) = (0.2126, 0.7152, 0.0722)$ are the Rec.709 luminance weights.

**Implementation:** See `buildSaturationMatrix()` at lines 68-95 in `ImageCanvas.tsx`:
```typescript
const buildSaturationMatrix = (saturation: number): number[] => {
  if (saturation === 1) return [1, 0, 0, 0, 1, 0, 0, 0, 1];
  
  const wR = 0.2126, wG = 0.7152, wB = 0.0722;
  const s = saturation;
  
  return [
    wR + (1 - wR) * s, wG * (1 - s), wB * (1 - s),
    wR * (1 - s), wG + (1 - wG) * s, wB * (1 - s),
    wR * (1 - s), wG * (1 - s), wB + (1 - wB) * s
  ];
};
```

### Proof of Correctness

**Claim:** The saturation transform (1) preserves luminance and (2) leaves gray pixels invariant.

**Proof:**

**(1) Luminance preservation:**

The luminance of the transformed pixel is:
$$L' = w_R R' + w_G G' + w_B B'$$

Substituting $\mathbf{r}' = L\mathbf{1}(1-s) + \mathbf{r} \cdot s$:
$$L' = w_R[L(1-s) + Rs] + w_G[L(1-s) + Gs] + w_B[L(1-s) + Bs]$$
$$= L(1-s)(w_R + w_G + w_B) + s(w_R R + w_G G + w_B B)$$
$$= L(1-s) \cdot 1 + s \cdot L = L$$

The luminance is preserved. 

**(2) Gray invariance:**

Let $\mathbf{r} = [g, g, g]^T$ be a gray pixel. Then:
$$L = w_R g + w_G g + w_B g = g(w_R + w_G + w_B) = g$$

Thus:
$$\mathbf{r}' = [g, g, g](1-s) + [g, g, g]s = [g, g, g]$$

Gray pixels remain unchanged for all $s$. 


**Why I chose Rec.709 weights:** I use $(0.2126, 0.7152, 0.0722)$ because these are the modern standard for sRGB displays and HDTV (Rec.709). They correctly account for how human eyes perceive different wavelengths—green appears brightest to us, so it gets the largest weight.



## 5. Vibrance Adjustment

**What I wanted:** An adaptive saturation that boosts dull colors more than already-saturated colors, preventing oversaturation of skin tones and vivid regions.

### Algorithm 5.1: Vibrance Transform

```
Input: pixel r = [R, G, B]ᵀ, vibrance factor v ∈ ℝ
Output: transformed pixel r'

1. maxC ← max(R, G, B)
2. minC ← min(R, G, B)
3. ŝ ← (maxC - minC) / maxC        // Saturation estimate
4. f ← 1 + v(1 - ŝ)                 // Adaptive factor
5. L ← 0.2126·R + 0.7152·G + 0.0722·B
6. r' ← [L, L, L]ᵀ + f(r - [L, L, L]ᵀ)
7. return clamp(r', 0, 255)
```

**Key insight:** Unlike saturation, the factor $f$ depends on each pixel's current saturation $\hat{s}(\mathbf{r})$, making this inherently per-pixel.

**Implementation:** See `applyVibrance())` at lines 194-210 in `ImageCanvas.tsx`:
```typescript
const applyVibrance = (rgb: RGB, vibrance: number): RGB => {
  if (vibrance === 0) return rgb;
  const R = rgb.r, G = rgb.g, B = rgb.b;
  const maxC = Math.max(R, G, B), minC = Math.min(R, G, B);
  const sEst = maxC === 0 ? 0 : (maxC - minC) / maxC;
  const f = 1 + vibrance * (1 - sEst);
  const gray = 0.2126 * R + 0.7152 * G + 0.0722 * B;
  if (R === G && G === B) return { r: R, g: G, b: B };
  return { 
    r: clamp(gray + (R - gray) * f), 
    g: clamp(gray + (G - gray) * f), 
    b: clamp(gray + (B - gray) * f) 
  };
};
```

### Proof of Non-Composability

**Theorem 5.2:** Vibrance cannot be expressed as a global affine transform (matrix + offset).

**Proof by contradiction:**

Assume vibrance can be expressed as $V(\mathbf{r}) = M\mathbf{r} + \mathbf{o}$ for some fixed $M$ and $\mathbf{o}$.

Consider two pixels:
- $\mathbf{r}_1 = [255, 0, 0]^T$ (saturated red): $\hat{s}_1 = 1$, so $f_1 = 1 + v(1-1) = 1$
- $\mathbf{r}_2 = [200, 195, 190]^T$ (desaturated): $\hat{s}_2 \approx 0.05$, so $f_2 = 1 + v(0.95) \approx 1 + 0.95v$

For $v = 1$, we have $f_1 = 1$ and $f_2 \approx 1.95$.

If vibrance were a linear transform, the ratio of how much each pixel changes would be constant:
$$\frac{\|\mathbf{r}'_1 - L_1\mathbf{1}\|}{\|\mathbf{r}_1 - L_1\mathbf{1}\|} = \frac{\|\mathbf{r}'_2 - L_2\mathbf{1}\|}{\|\mathbf{r}_2 - L_2\mathbf{1}\|}$$

But by construction:
$$\frac{\|\mathbf{r}'_1 - L_1\mathbf{1}\|}{\|\mathbf{r}_1 - L_1\mathbf{1}\|} = f_1 = 1 \neq 1.95 \approx f_2 = \frac{\|\mathbf{r}'_2 - L_2\mathbf{1}\|}{\|\mathbf{r}_2 - L_2\mathbf{1}\|}$$

This contradicts linearity. Therefore, vibrance cannot be a global affine transform. 




## 6. Hue Rotation

**What I wanted:** A transform that rotates colors around the gray axis in RGB space, changing hue while preserving brightness and leaving grays unchanged.

### Algorithm 6.1: Hue Rotation

```
Input: pixel r = [R, G, B]ᵀ, angle θ (in degrees)
Output: rotated pixel r'

1. Convert θ to radians: θ_rad ← θ · π/180
2. Compute rotation matrix R(θ) using Rodrigues' formula
3. r' ← R(θ) · r
4. return clamp(r', 0, 255)
```

**Mathematical form:** I use Rodrigues' rotation formula for rotation by angle $\theta$ around the normalized gray axis $\mathbf{u} = \frac{1}{\sqrt{3}}[1, 1, 1]^T$:

$$R(\theta) = \cos\theta \cdot I + (1-\cos\theta)(\mathbf{u}\mathbf{u}^T) + \sin\theta \cdot [\mathbf{u}]_\times$$

After simplification, this becomes:

$$R(\theta) = \begin{bmatrix}
c + \frac{1-c}{3} & \frac{1-c}{3} - \frac{s}{\sqrt{3}} & \frac{1-c}{3} + \frac{s}{\sqrt{3}} \\
\frac{1-c}{3} + \frac{s}{\sqrt{3}} & c + \frac{1-c}{3} & \frac{1-c}{3} - \frac{s}{\sqrt{3}} \\
\frac{1-c}{3} - \frac{s}{\sqrt{3}} & \frac{1-c}{3} + \frac{s}{\sqrt{3}} & c + \frac{1-c}{3}
\end{bmatrix}$$

where $c = \cos\theta$ and $s = \sin\theta$.

**Implementation:** See `buildHueMatrix()` at lines 98-123 in `ImageCanvas.tsx`:
```typescript
const buildHueMatrix = (value: number): number[] => {
  if (value === 0) return [1, 0, 0, 0, 1, 0, 0, 0, 1];
  
  const angle = (value * Math.PI) / 180;
  const cosA = Math.cos(angle);
  const sinA = Math.sin(angle);
  
  return [
    cosA + (1 - cosA) / 3,
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    cosA + 1/3 * (1 - cosA),
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) - Math.sqrt(1/3) * sinA,
    1/3 * (1 - cosA) + Math.sqrt(1/3) * sinA,
    cosA + 1/3 * (1 - cosA)
  ];
};
```

### Proof of Gray Invariance

**Claim:** Any gray pixel $\mathbf{g} = g\mathbf{u}\sqrt{3} = [g, g, g]^T$ remains unchanged under hue rotation.

**Proof:**

By definition of rotation, any vector parallel to the rotation axis is unchanged. Since $\mathbf{g} = g[1,1,1]^T = g\sqrt{3} \cdot \mathbf{u}$ is parallel to $\mathbf{u}$:

$$R(\theta)\mathbf{g} = R(\theta)(g\sqrt{3}\mathbf{u}) = g\sqrt{3} \cdot R(\theta)\mathbf{u} = g\sqrt{3} \cdot \mathbf{u} = \mathbf{g}$$

The last equality follows because rotating a vector around itself leaves it unchanged. 

Because $R(\theta)$ is an orthogonal matrix: $R(\theta)^T R(\theta) = I$, hue rotation preserves distances and angles in RGB space.



## 7. Transform Composition

I realized that brightness, contrast, saturation, and hue are all affine transforms (matrix + offset). By composing them algebraically before processing pixels, I can apply multiple transforms with a single matrix multiplication per pixel.

### Algorithm 7.1: Affine Transform Composition

```
Input: List of transforms T₁, T₂, ..., Tₙ where Tᵢ = (Mᵢ, oᵢ)
Output: Composed transform T_composed = (M_composed, o_composed)

1. M ← M₁
2. o ← o₁
3. for i = 2 to n:
4.   M ← Mᵢ · M          // Matrix multiplication
5.   o ← Mᵢ · o + oᵢ      // Transform offset through matrix
6. return (M, o)
```

**Mathematical justification:** Composing two affine transforms:
$$\mathbf{r}'' = M_2(M_1\mathbf{r} + \mathbf{o}_1) + \mathbf{o}_2 = (M_2 M_1)\mathbf{r} + (M_2\mathbf{o}_1 + \mathbf{o}_2)$$

So the composed transform is $(M_2 M_1, M_2\mathbf{o}_1 + \mathbf{o}_2)$.

**Implementation:** See `composeAffineTransforms()` at lines 235-286 in `ImageCanvas.tsx`:
```typescript
const composeAffineTransforms = (
  transforms: Array<{ matrix: number[]; offset: number[] }>
): { matrix: number[]; offset: number[] } => {
  if (transforms.length === 0) {
    return { matrix: [1, 0, 0, 0, 1, 0, 0, 0, 1], offset: [0, 0, 0] };
  }
  
  if (transforms.length === 1) {
    return transforms[0];
  }
  
  let resultMatrix = [...transforms[0].matrix];
  let resultOffset = [...transforms[0].offset];
  
  for (let i = 1; i < transforms.length; i++) {
    const M2 = transforms[i].matrix;
    const o2 = transforms[i].offset;
    
    // Matrix multiplication: M_composed = M2 * M1
    const newMatrix = [
      M2[0] * resultMatrix[0] + M2[1] * resultMatrix[3] + M2[2] * resultMatrix[6],
      M2[0] * resultMatrix[1] + M2[1] * resultMatrix[4] + M2[2] * resultMatrix[7],
      M2[0] * resultMatrix[2] + M2[1] * resultMatrix[5] + M2[2] * resultMatrix[8],
      M2[3] * resultMatrix[0] + M2[4] * resultMatrix[3] + M2[5] * resultMatrix[6],
      M2[3] * resultMatrix[1] + M2[4] * resultMatrix[4] + M2[5] * resultMatrix[7],
      M2[3] * resultMatrix[2] + M2[4] * resultMatrix[5] + M2[5] * resultMatrix[8],
      M2[6] * resultMatrix[0] + M2[7] * resultMatrix[3] + M2[8] * resultMatrix[6],
      M2[6] * resultMatrix[1] + M2[7] * resultMatrix[4] + M2[8] * resultMatrix[7],
      M2[6] * resultMatrix[2] + M2[7] * resultMatrix[5] + M2[8] * resultMatrix[8]
    ];
    
    // Offset transformation: o_composed = M2 * o1 + o2
    const newOffset = [
      M2[0] * resultOffset[0] + M2[1] * resultOffset[1] + M2[2] * resultOffset[2] + o2[0],
      M2[3] * resultOffset[0] + M2[4] * resultOffset[1] + M2[5] * resultOffset[2] + o2[1],
      M2[6] * resultOffset[0] + M2[7] * resultOffset[1] + M2[8] * resultOffset[2] + o2[2]
    ];
    
    resultMatrix = newMatrix;
    resultOffset = newOffset;
  }
  
  return { matrix: resultMatrix, offset: resultOffset };
};
```

### Proof of Correctness

For transforms $T_1, T_2, \ldots, T_n$, Algorithm 7.1 produces a transform $T$ such that:
$$T(\mathbf{r}) = T_n(T_{n-1}(\cdots T_2(T_1(\mathbf{r})) \cdots))$$

**Proof by induction:**

*Base case ($n=1$):* Trivial—the single transform is returned as-is.

*Inductive step:* Assume correctness for $n-1$ transforms. Let $(M', \mathbf{o}')$ be the composition of $T_1, \ldots, T_{n-1}$. After iteration $n$:
$$M = M_n M', \quad \mathbf{o} = M_n\mathbf{o}' + \mathbf{o}_n$$

Applying this to pixel $\mathbf{r}$:
$$T(\mathbf{r}) = M\mathbf{r} + \mathbf{o} = M_n M'\mathbf{r} + M_n\mathbf{o}' + \mathbf{o}_n$$
$$= M_n(M'\mathbf{r} + \mathbf{o}') + \mathbf{o}_n = T_n(T'(\mathbf{r}))$$

where $T'$ is the composition of $T_1, \ldots, T_{n-1}$. By the inductive hypothesis, this equals the full composition. 


## 8. Whites and Blacks (Tone Curve)

**What I wanted:** Parametric tone curve adjustments that selectively affect highlights (whites) or shadows (blacks) while preserving midtone detail and maintaining smooth transitions without banding artifacts. These adjustments should move pixels along the gray axis (preserving hue) with strength that varies smoothly based on luminance.

### Algorithm 8.1: Whites (Highlights) Transform

```
Input: pixel r = [R, G, B]ᵀ, whites adjustment w ∈ ℝ
Output: transformed pixel r' = [R', G', B']ᵀ

1. L ← (0.299·R + 0.587·G + 0.114·B) / 255    // Normalized Rec.601 gamma luminance
2. t ← clamp((L - 0.4) / (0.8 - 0.4), 0, 1)   // Map [0.4, 0.8] → [0, 1]
3. s(t) ← t²·(3 - 2t)                          // Smoothstep weighting function
4. Δ ← w · s(t)                                 // Adjustment scaled by weight
5. R' ← clamp(R + Δ, 0, 255)
6. G' ← clamp(G + Δ, 0, 255)
7. B' ← clamp(B + Δ, 0, 255)
8. return [R', G', B']ᵀ
```

**Affine form:** $\mathbf{r}' = I\mathbf{r} + \Delta(L)\mathbf{1}$ where $I$ is the $3 \times 3$ identity matrix, $\mathbf{1} = [1, 1, 1]^T$, and $\Delta(L) = w \cdot s(t(L))$ depends on the pixel's luminance $L$ through the smoothstep function $s(t)$.

**Implementation:** See `applyWhites()` at lines 266-279 in `ImageCanvas.tsx`:

```typescript
const smoothstep = (edge0: number, edge1: number, x: number): number => {
  const t = Math.max(0, Math.min(1, (x - edge0) / (edge1 - edge0)));
  return t * t * (3 - 2 * t);
};

const applyWhites = (rgb: RGB, value: number): RGB => {
  if (value === 0) return rgb;
  const luminance = (0.299 * rgb.r + 0.587 * rgb.g + 0.114 * rgb.b) / 255;
  const weight = smoothstep(0.4, 0.8, luminance);
  const adjustment = value * weight;
  return {
    r: clamp(rgb.r + adjustment),
    g: clamp(rgb.g + adjustment),
    b: clamp(rgb.b + adjustment)
  };
};
```

### Algorithm 8.2: Blacks (Shadows) Transform

```
Input: pixel r = [R, G, B]ᵀ, blacks adjustment b ∈ ℝ
Output: transformed pixel r' = [R', G', B']ᵀ

1. L ← (0.299·R + 0.587·G + 0.114·B) / 255    // Normalized Rec.601 gamma luminance
2. t ← clamp((L - 0.8) / (0.2 - 0.8), 0, 1)   // Map [0.8, 0.2] → [0, 1] (inverted)
3. s(t) ← t²·(3 - 2t)                          // Smoothstep weighting function
4. Δ ← b · s(t)                                 // Adjustment scaled by weight
5. R' ← clamp(R + Δ, 0, 255)
6. G' ← clamp(G + Δ, 0, 255)
7. B' ← clamp(B + Δ, 0, 255)
8. return [R', G', B']ᵀ
```

**Affine form:** $\mathbf{r}' = I\mathbf{r} + \Delta(L)\mathbf{1}$ where $\Delta(L) = b \cdot s(t(L))$ with $t(L)$ mapping the luminance range $[0.8, 0.2]$ to $[0, 1]$ (inverted compared to whites).

**Implementation:** See `applyBlacks()` at lines 281-294 in `ImageCanvas.tsx`:

```typescript
const applyBlacks = (rgb: RGB, value: number): RGB => {
  if (value === 0) return rgb;
  const luminance = (0.299 * rgb.r + 0.587 * rgb.g + 0.114 * rgb.b) / 255;
  const weight = smoothstep(0.8, 0.2, luminance);
  const adjustment = value * weight;
  return {
    r: clamp(rgb.r + adjustment),
    g: clamp(rgb.g + adjustment),
    b: clamp(rgb.b + adjustment)
  };
};
```

### Proof of Correctness

**Claim 1 (Whites):** Pixels with $L \leq 0.4$ remain unchanged (shadows are fixed points).

**Proof:**
For $L \leq 0.4$, we have $t = \max(0, (L - 0.4) / 0.4) = 0$ (since $L - 0.4 \leq 0$). Therefore $s(0) = 0^2 \cdot (3 - 2 \cdot 0) = 0$, so $\Delta = w \cdot 0 = 0$. Hence $\mathbf{r}' = I\mathbf{r} + 0 \cdot \mathbf{1} = \mathbf{r}$. ∎

**Claim 2 (Whites):** Pixels with $L \geq 0.8$ receive uniform adjustment (highlights are uniformly translated).

**Proof:**
For $L \geq 0.8$, we have $t = \min(1, (L - 0.4) / 0.4) = 1$ (since $(L - 0.4) / 0.4 \geq 1$). Therefore $s(1) = 1^2 \cdot (3 - 2 \cdot 1) = 1$, so $\Delta = w \cdot 1 = w$ (constant). For any two pixels $\mathbf{r}_1, \mathbf{r}_2$ with $L_1, L_2 \geq 0.8$:
$$\mathbf{r}'_1 - \mathbf{r}'_2 = (I\mathbf{r}_1 + w\mathbf{1}) - (I\mathbf{r}_2 + w\mathbf{1}) = I(\mathbf{r}_1 - \mathbf{r}_2) = \mathbf{r}_1 - \mathbf{r}_2$$
The relative differences are preserved, and both receive the same absolute offset $w$ along the gray axis. ∎

**Claim 3 (Smoothness):** The smoothstep function $s(t) = t^2(3 - 2t)$ provides $C^1$ continuity at the transition boundaries.

**Proof:**
We have $s(0) = 0$ and $s(1) = 1$. The derivative is $s'(t) = 6t - 6t^2 = 6t(1 - t)$, so $s'(0) = 0$ and $s'(1) = 0$. Since both the function value and its derivative vanish at the endpoints, the composition $\Delta(L) = w \cdot s(t(L))$ is $C^1$ continuous across the transition regions, eliminating kinks and banding artifacts. ∎

**Claim 4 (Hue preservation):** Both transforms preserve hue and chroma, affecting only lightness.

**Proof:**
Since $\mathbf{r}' = \mathbf{r} + \Delta\mathbf{1}$, we have:
$$(R', G', B') = (R + \Delta, G + \Delta, B + \Delta) = (R, G, B) + \Delta(1, 1, 1)$$
Adding a multiple of $(1, 1, 1)$ moves the pixel along the gray diagonal in RGB space. This preserves the direction of the color vector relative to the gray axis, maintaining hue and chroma while only changing perceived lightness. ∎

**Claim 5 (Blacks):** Analogous properties hold for blacks with inverted luminance mapping.

**Proof:**
For blacks, $t(L) = \text{clamp}((L - 0.8) / (0.2 - 0.8), 0, 1)$ maps $L \geq 0.8$ to $t = 0$ (highlights unchanged) and $L \leq 0.2$ to $t = 1$ (deep shadows receive full effect). The same smoothstep function ensures $C^1$ continuity, and the same gray-axis translation preserves hue. ∎

*Note:* I use Rec.601 weights $(0.299, 0.587, 0.114)$ for gamma-space luminance, consistent with saturation and vibrance. The smoothstep function is a standard technique in computer graphics for creating smooth transitions without discontinuities.



## 9. Convolution Filters

**What I wanted:** Spatial image filters that operate on local neighborhoods of pixels, enabling effects like blurring, sharpening, edge detection, and denoising. These filters should support various kernel types, padding modes, and be efficiently computable per-pixel while processing each RGB channel independently.

### Algorithm 9.1: General 2D Convolution

```
Input: Image I, kernel K of size (2k+1) × (2k+1), padding mode p ∈ {zero, edge, reflect}
Output: Filtered image I'

For each pixel (x, y) in I:
1. Initialize accumulator acc ← [0, 0, 0] for [R, G, B]
2. For each offset (i, j) where -k ≤ i, j ≤ k:
3.   (x', y') ← padIndex(x + i, y + j, p)  // Handle boundary via padding
4.   if (x', y') is valid:
5.     acc ← acc + K[i+k, j+k] · I(x', y')
6. I'(x, y) ← clamp(acc, 0, 255)
```

**Mathematical form:** For channel $c \in \{R, G, B\}$ at pixel $(x, y)$:
$$I'_c(x,y) = \sum_{i=-k}^k \sum_{j=-k}^k K[i,j] \cdot I_c(x+i, y+j)$$
Padding modes: zero, edge, reflect. Dilation spaces taps; stride can subsample outputs (ImageLab fills skipped pixels with nearest previous sample). Per-channel unless explicitly run on luminance.

**Implementation:** See `convolveAtPixel()` and `convolveImageData()` in `convolution.ts`:

```typescript
export function convolveAtPixel(
  source: ImageData,
  x: number,
  y: number,
  kernel: number[][],
  params: ConvolutionCommonParams & { perChannel?: boolean }
): [number, number, number] {
  const { width, height, data } = source;
  const k = kernel;
  const kSize = k.length;
  const kHalf = Math.floor(kSize / 2);
  const padding: PaddingMode = params.padding ?? 'edge';
  const perChannel = params.perChannel ?? true;

  let rAcc = 0, gAcc = 0, bAcc = 0;
  for (let ky = 0; ky < kSize; ky++) {
    for (let kx = 0; kx < kSize; kx++) {
      const ix = x + (kx - kHalf);
      const iy = y + (ky - kHalf);
      let sx = padIndex(ix, width, padding);
      let sy = padIndex(iy, height, padding);
      const w = k[ky][kx];
      if (sx === -1 || sy === -1) continue; // zero padding
      const idx = (sy * width + sx) * 4;
      const R = data[idx], G = data[idx + 1], B = data[idx + 2];
      if (perChannel) {
        rAcc += R * w;
        gAcc += G * w;
        bAcc += B * w;
      } else {
        const gray = 0.299 * R + 0.587 * G + 0.114 * B;
        rAcc += gray * w;
        gAcc += gray * w;
        bAcc += gray * w;
      }
    }
  }
  return [clamp255(rAcc), clamp255(gAcc), clamp255(bAcc)];
}
```

### Algorithm 9.2: Blur (Gaussian and Box)

Gaussian kernel: $K[i,j] = e^{-(i^2+j^2)/(2\sigma^2)} / Z$ where $Z = \sum_{i,j} e^{-(i^2+j^2)/(2\sigma^2)}$ normalizes so $\sum K = 1$.

Box kernel: $K[i,j] = 1/n^2$ for all entries in an $n \times n$ kernel.

**Implementation:** See `gaussianKernel()` and `boxKernel()` in `convolution.ts`, and `applyBlur()`:

```typescript
export function gaussianKernel(size: 3 | 5 | 7, sigma?: number): number[][] {
  const s = sigma ?? (size === 3 ? 0.85 : size === 5 ? 1.2 : 1.6);
  const half = Math.floor(size / 2);
  const k: number[][] = [];
  let sum = 0;
  for (let y = -half; y <= half; y++) {
    const row: number[] = [];
    for (let x = -half; x <= half; x++) {
      const v = Math.exp(-(x * x + y * y) / (2 * s * s));
      row.push(v);
      sum += v;
    }
    k.push(row);
  }
  // normalize
  for (let y = 0; y < size; y++) {
    for (let x = 0; x < size; x++) k[y][x] /= sum;
  }
  return k;
}

export function applyBlur(imageData: ImageData, params: BlurParams): ImageData {
  const kernel = params.kind === 'gaussian' 
    ? gaussianKernel(params.size, params.sigma) 
    : boxKernel(params.size);
  return convolveImageData(imageData, kernel, { 
    stride: params.stride ?? 1, 
    padding: params.padding ?? 'edge', 
    perChannel: true 
  });
}
```

### Algorithm 9.3: Sharpen (Unsharp Mask)
Start from identity δ (center=1) and subtract a blurred kernel B: $K = δ + a(δ - B)$. Center becomes $1+a$, neighbors become $-a·B_{ij}$. **Proof:** Decomposes as original + a·(original−blur); this is high-boost filtering that adds back high frequencies.
- Laplacian/Edge-Enhance matrix:
$$
\begin{bmatrix}
0 & -a & 0 \\
-a & 1+4a & -a \\
0 & -a & 0
\end{bmatrix}
$$
Adding this output to the original amplifies second derivatives; positive a sharpens, negative a smooths.

### Algorithm 9.4: Edge Detection (Sobel/Prewitt)
Sobel matrices:
$K_x = \begin{bmatrix}-1&0&1\\-2&0&2\\-1&0&1\end{bmatrix},\quad K_y = \begin{bmatrix}-1&-2&-1\\0&0&0\\1&2&1\end{bmatrix}$

Prewitt matrices:
$K_x = \begin{bmatrix}-1&0&1\\-1&0&1\\-1&0&1\end{bmatrix},\quad K_y = \begin{bmatrix}-1&-1&-1\\0&0&0\\1&1&1\end{bmatrix}$


### 9.4 Denoise
- Mean filter: box blur followed by blend `out = (1−k)·original + k·mean`. Matrix view: blur is convolution with box kernel (sums to 1), then affine blend preserves DC. For k=1 it is pure mean; for k=0 it is identity.
- Median filter: collects each channel’s neighborhood, sorts, returns the median. Not expressible as a fixed matrix (non-linear). 

## 10. Processing Pipeline

Transform categories:
- Affine (batchable): brightness, contrast, saturation, hue → same matrix/offset for all pixels.
- Per-pixel non-linear: vibrance, whites, blacks → depend on each pixel’s luminance/saturation; not batchable.
- Convolution-backed: blur, sharpen, edge, denoise, custom → spatial kernels; not batchable with affine color matrices.

### Algorithm 10.1: Smart Batching Pipeline

```
Input: Pixel array P, ordered transform list L
Output: Transformed pixel array P'

1. i ← 0
2. while i < |L|:
3.   batch ← []
4.   while i < |L| and L[i] is affine:
5.     batch.append(L[i]); i++
6.   if batch ≠ ∅:
7.     (M, o) ← composeAffineTransforms(batch)
8.     for each pixel p in P: p ← M · p + o
9.   if i = |L|: break
10.  if L[i] is per-pixel non-linear (vibrance / whites / blacks):
11.     for each pixel p in P: p ← L[i].apply(p)
12.  else if L[i] is convolution-backed:
13.     P ← runConvolution(P, L[i])   // blur/sharpen/edge/denoise/custom
14.  i++
15. return P
```

- Affine steps commute via composition, so batching reduces per-pixel work to one matrix multiply + offset per batch.
- Non-linear per-pixel steps depend on each pixel’s state, so they must run explicitly on every pixel.
- Convolution steps use spatial kernels and cannot be absorbed into the affine color matrix; they execute in-order where they appear.
- The algorithm preserves the user-specified order across all categories while minimizing redundant passes for the affine subset.



# References

Bezryadin, S., Bourov, P., & Ilinih, D. (2007, January). Brightness calculation in digital image processing. In International symposium on technologies for digital photo fulfillment (Vol. 1, pp. 10-15). Society for Imaging Science and Technology. https://library.imaging.org/admin/apis/public/api/ist/website/downloadArticle/tdpf/1/1/art00005

Huamán, A. (n.d.). Changing the contrast and brightness of an image!. OpenCV. https://docs.opencv.org/4.x/d3/dc1/tutorial_basic_linear_transform.html

Lisitsa, N. (2024, October 10). Transforming colors with matrices. lisyarus blog. https://lisyarus.github.io/blog/posts/transforming-colors-with-matrices.html

McNair, M. (n.d.). Color Correction. https://maximmcnair.com/p/webgl-color-correction 

Meyer, M. (2009, January 19). Analyzing photoshop vibrance and saturation. Alaska Photographer Mark Meyer. https://www.photo-mark.com/notes/analyzing-photoshop-vibrance-and-saturation/ 

Rodrigues’ rotation formula. Wikipedia. https://en.wikipedia.org/wiki/Rodrigues%27_rotation_formula 

Relative luminance. Wikipedia. https://en.wikipedia.org/wiki/Relative_luminance 

sRGB. Wikipedia. https://en.wikipedia.org/wiki/Relative_luminance  https://en.wikipedia.org/wiki/SRGB

Reinhard, E., Stark, M., Shirley, P., & Ferwerda, J. (2023). Photographic tone reproduction for digital images. In Seminal Graphics Papers: Pushing the Boundaries, Volume 2 (pp. 661-670). https://www-old.cs.utah.edu/docs/techreports/2002/pdf/UUCS-02-001.pdf

Vicapow. (n.d.). Image kernels explained visually. Explained Visually. https://setosa.io/ev/image-kernels/ 

GeeksforGeeks. (2025a, July 23). Types of convolution kernels. https://www.geeksforgeeks.org/deep-learning/types-of-convolution-kernels/ 

Kernel (image processing) (2025, October 5). Wikipedia. https://en.wikipedia.org/wiki/Kernel_(image_processing) 